In [1]:
import sys
from pathlib import Path
project_path = str(Path.cwd().parent)
sys.path.append(project_path)

In [8]:
import sys

from bs4 import BeautifulSoup
from apps.tabs import tab1, tab2, tab3, tab4, tab5
from agents import get_user_agents

from schemas.general import GeneralCreateSchema
from schemas.subscriber import SubscriberCreateSchema
from schemas.financial import FinancialCreateSchema
from schemas.shareholder import ShareholderCreateSchema
from schemas.prediction import PredictionCreateSchema

if __name__ == "__main__":
    import a_tags
    import settings
    import company_code
    import time

    count = 0
    url = f"{settings.IPO_URL}/view_pg"
    # company_codes = company_code.scrape_company_codes()

    # for code in company_codes:
    for code in ["B202010131"]:
        general_result = {}

        categories = a_tags.scrape_categories(url, code)

        for idx, category in enumerate(categories,1):
            if category == 1:
                result = tab1.scrape_ipostock(code)
                if result:
                    general_result.update(result)
            if category == 2:
                result, shareholder_results = tab2.scrape_ipostock(code)
                if result:
                    general_result.update(result)

            if category == 3:
                financial_results = tab3.scrape_ipostock(code)
            if category == 4:
                result, subscriber_results = tab4.scrape_ipostock(code)
                if result:
                    general_result.update(result)
            if category == 5:
                prediction_results, general_result5 = tab5.scrape_ipostock(code)
                if general_result5:
                    general_result.update(general_result5)
                
        general = GeneralCreateSchema(**general_result)
        shareholders = [
            ShareholderCreateSchema(**shareholder) for shareholder in shareholder_results or []
        ]
        subscribers = [
            SubscriberCreateSchema(**subscriber) for subscriber in subscriber_results or []
        ]
        financials = [FinancialCreateSchema(**financial) for financial in financial_results or []]
        predictions = [
            PredictionCreateSchema(**prediction) for prediction in prediction_results or []
        ]
        count += 1
        print(count, code)

1 B202010131


In [3]:
from db import models

# MariaDB
DEV_DB = {
    "HOST": "ipolistingdb.civxowlytnik.ap-northeast-2.rds.amazonaws.com",
    "USER": "admin",
    "PASSWORD": "dara100400!",
    "PORT": int("3306"),
    "DB_NAME": "trustinv",
}
DB_URL = f'mariadb+pymysql://{DEV_DB["USER"]}:{DEV_DB["PASSWORD"]}@{DEV_DB["HOST"]}:{DEV_DB["PORT"]}/{DEV_DB["DB_NAME"]}'


In [4]:
from contextlib import contextmanager


@contextmanager
def session_scope(Session):
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()


In [5]:
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [10]:
class DBManager:
    def __init__(self, *args, **kwargs):
        self.engine = create_engine(DB_URL)
        self.Session = sessionmaker(bind=self.engine)

    def delete(self):
        # "상장철회"일 경우 컬럼 필드 값을 변경함.
        pass

    def create(self):
        pass

    def create_n_update(self):
        """
        1) 데이터 존재여부를 확인.
            - 상장철회 된 데이터는 가져 올 필요가 없음.
        """
        with session_scope(self.Session) as session:
            print(general.ci_name)
            # Check if a company with the same name already exists in the database
            existing_company = (
                session.query(models.CompanyInfoGeneral)
                .filter_by(ci_name=general.ci_name)
                .first()
            )

            if existing_company is None:
                # Add the new company to the database
                session.begin_nested()
                company = models.CompanyInfoGeneral(**general.dict())
        
                shareholder_model_instances = [models.CompanyInfoShareholder(shareholder.dict().update(company=company)) for shareholder in shareholders]
                prediction_model_instances = [models.CompanyInfoPrediction(prediction.dict().update(company=company)) for prediction in predictions]
                subscriber_model_instances = [models.CompanyInfoSubscriber(subscriber.dict().update(company=company)) for subscriber in subscribers]
                financials_model_instances = [models.CompanyInfoFinancial(financial.dict().update(company=company)) for financial in financials]
                session.bulk_save_objects([company] + shareholder_model_instances +prediction_model_instances +subscriber_model_instances +financials_model_instances)
                session.commit()
                # session.add(cig)
            
            else:
                # Check if any of the values have changed
                update_required = False
                for key, value in existing_company.dict().items():
                    if key in ["_sa_instance_state", "id"] or value == getattr(
                        existing_company, key
                    ):
                        continue
                    setattr(existing_company, key, value)
                    update_required = True
                if update_required:
                    session.commit()

    def read(self):
        pass

session = DBManager()
session.create_n_update()


래몽래인


AttributeError: 'NoneType' object has no attribute 'ci_name'

In [49]:
class DBManager:
    def __init__(self, *args, **kwargs):
        self.engine = create_engine(DB_URL)
        self.Session = sessionmaker(bind=self.engine)

    def delete(self):
        # "상장철회"일 경우 컬럼 필드 값을 변경함.
        pass

    def create(self):
        pass

    def create_n_update(self):
        """
        1) 데이터 존재여부를 확인.
            - 상장철회 된 데이터는 가져 올 필요가 없음.
        """
        with session_scope(self.Session) as session:
            print(general.ci_name)
            # Check if a company with the same name already exists in the database
            existing_company = (
                session.query(models.CompanyInfoGeneral)
                .filter_by(ci_name=general.ci_name)
                .first()
            )
            print(existing_company)
            if existing_company is None:
                # Add the new company to the database
                session.begin_nested()
                company = models.CompanyInfoGeneral(**general.dict())
                session.add(company)
                session.flush()
                session.commit()
                session.bulk_insert_mappings(models.CompanyInfoShareholder,[ {**shareholder.dict(), "ci_idx":company.ci_idx} for shareholder in shareholders])
                session.bulk_insert_mappings(models.CompanyInfoShareholder,[ {**shareholder.dict(), "ci_idx":company.ci_idx} for shareholder in shareholders])
                session.bulk_insert_mappings(models.CompanyInfoPrediction,[ {**prediction.dict(), "ci_idx":company.ci_idx} for prediction in predictions])
                session.bulk_insert_mappings(models.CompanyInfoSubscriber,[ {**subscriber.dict(), "ci_idx":company.ci_idx} for subscriber in subscribers])
                session.bulk_insert_mappings(models.CompanyInfoFinancial,[ {**financial.dict(), 'ci_idx': company.ci_idx} for financial in financials])
                session.commit()
                print('커밋완료')
                # session.add(cig)
            
            # else:
            #     # Check if any of the values have changed
            #     update_required = False
            #     for key, value in general_instance.dict().items():
            #         if key in ["_sa_instance_state", "id"] or value == getattr(
            #             existing_company, key
            #         ):
            #             continue
            #         setattr(existing_company, key, value)
            #         update_required = True
            #     if update_required:
            #         session.commit()

    def read(self):
        pass

session = DBManager()
session.create_n_update()


툴젠
None
커밋완료


In [11]:
class DBManager:
    def __init__(self, *args, **kwargs):
        self.engine = create_engine(DB_URL)
        self.Session = sessionmaker(bind=self.engine)

    def read(self, **kwargs):
        general = kwargs['general']
        shareholders = kwargs['shareholders']
        predictions = kwargs['predictions']
        subscribers = kwargs['subscribers']
        financials = kwargs['financials']

        result = session.query(models.CompanyInfoGeneral)\
            .filter(and_(models.CompanyInfoGeneral.ci_name == ci_name, 
                        models.CompanyInfoGeneral.column_name == value))\
            .all()

    def delete(self):
        # "상장철회"일 경우 컬럼 필드 값을 변경함.
        pass

    def create(self, **kwargs):
        
        general = kwargs['general']
        shareholders = kwargs['shareholders']
        predictions = kwargs['predictions']
        subscribers = kwargs['subscribers']
        financials = kwargs['financials']

        with session_scope(self.Session) as session:
            session.begin_nested()
            company = models.CompanyInfoGeneral(**general.dict())
            session.add(company)
            session.flush()
            session.commit()
            session.bulk_insert_mappings(models.CompanyInfoShareholder,[ {**shareholder.dict(), "ci_idx":company.ci_idx} for shareholder in shareholders])
            session.bulk_insert_mappings(models.CompanyInfoPrediction,[ {**prediction.dict(), "ci_idx":company.ci_idx} for prediction in predictions])
            session.bulk_insert_mappings(models.CompanyInfoSubscriber,[ {**subscriber.dict(), "ci_idx":company.ci_idx} for subscriber in subscribers])
            session.bulk_insert_mappings(models.CompanyInfoFinancial,[ {**financial.dict(), 'ci_idx': company.ci_idx} for financial in financials])
            session.commit()
            return True

    def create_n_update(self, **kwargs):
        """
        1) 데이터 존재여부를 확인.
            - 상장철회 된 데이터는 가져 올 필요가 없음.
        """
        general = kwargs.get('general', {})
        shareholders = kwargs.get('shareholders', [])
        predictions = kwargs.get('predictions', [])
        subscribers = kwargs.get('subscribers', [])
        financials = kwargs.get('financials', [])
        success = self.create(
            general=general,
            shareholders=shareholders,
            predictions=predictions,
            subscribers=subscribers,
            financials=financials,
        )


    def read(self):
        pass

session = DBManager()
session.create_n_update()


래몽래인
None


In [56]:
for i in shareholders:
    print(i.dict())

{'ci_category': 1, 'ci_category_name': '최대주주', 'ci_normal_stocks': 2096826, 'ci_first_stocks': 0, 'ci_share_rate': 33.56, 'ci_protection_date': '상장 후 2년', 'ci_datetime': datetime.datetime(2023, 1, 18, 15, 49, 57, 581505)}
{'ci_category': 1, 'ci_category_name': '최대주주', 'ci_normal_stocks': 19817, 'ci_first_stocks': 0, 'ci_share_rate': 0.32, 'ci_protection_date': '상장 후 1년', 'ci_datetime': datetime.datetime(2023, 1, 18, 15, 49, 57, 581505)}
{'ci_category': 1, 'ci_category_name': '벤처금융', 'ci_normal_stocks': 1145000, 'ci_first_stocks': 0, 'ci_share_rate': 18.32, 'ci_protection_date': '상장 후 1개월', 'ci_datetime': datetime.datetime(2023, 1, 18, 15, 49, 57, 581505)}
{'ci_category': 1, 'ci_category_name': '상장주선인의무인수분', 'ci_normal_stocks': 60000, 'ci_first_stocks': 0, 'ci_share_rate': 0.96, 'ci_protection_date': '상장 후 6개월', 'ci_datetime': datetime.datetime(2023, 1, 18, 15, 49, 57, 581505)}
{'ci_category': 2, 'ci_category_name': '벤처금융', 'ci_normal_stocks': 657215, 'ci_first_stocks': 0, 'ci_share_rat

In [54]:
predictions#,subscribers,financials,

[PredictionCreateSchema(ci_price='가격미제시 (M.O)', ci_incidence=26, ci_incidence_specific_gravity=1.65, ci_participation=23340000, ci_participation_specific_gravity=1.68, ci_datetime=datetime.datetime(2023, 1, 18, 15, 49, 57, 583269)),
 PredictionCreateSchema(ci_price='13,000원 이상', ci_incidence=1549, ci_incidence_specific_gravity=98.29, ci_participation=1366899333, ci_participation_specific_gravity=98.26, ci_datetime=datetime.datetime(2023, 1, 18, 15, 49, 57, 583269)),
 PredictionCreateSchema(ci_price='11,500원 이상 13,000원 미만', ci_incidence=1, ci_incidence_specific_gravity=0.06, ci_participation=874000, ci_participation_specific_gravity=0.06, ci_datetime=datetime.datetime(2023, 1, 18, 15, 49, 57, 583269)),
 PredictionCreateSchema(ci_price='11,500원 미만', ci_incidence=0, ci_incidence_specific_gravity=0.0, ci_participation=0, ci_participation_specific_gravity=0.0, ci_datetime=datetime.datetime(2023, 1, 18, 15, 49, 57, 583269))]

In [57]:
general

GeneralCreateSchema(ci_market_separation='코넥스', ci_progress='', ci_name='툴젠', ci_code='199800', ci_logo='', ci_logo_name='', ci_keyword1='', ci_keyword2='', ci_keyword3='', ci_keyword4='', ci_list_type='', ci_review_c_date='2021.09.14', ci_review_a_date='2021.11.04', ci_face_value=500, ci_ceo='이병화', ci_tel='02-873-8168', ci_homepage='www.toolgen.com', ci_establishment_date='1999/10/08', ci_company_separation='이노비즈&벤처', ci_brn='108-81-18555', ci_settlement_month=12, ci_worker_cnt=66, ci_industries='자연과학 및 공학 연구개발업', ci_important_product='유전자교정 제품 및 서비스', ci_stocks_separation='통일주권', ci_lead_manager='한국투자증권', ci_address='서울특별시 금천구 가산디지털1로 219(가산동) 벽산디지털밸리 6차 1204,1205,1206', ci_turnover=7.13, ci_before_corporate_tax=-148.82, ci_net_profit=-148.82, ci_capital=298.91, ci_largest_shareholder='(주)제넥신', ci_largest_shareholder_rate=0.16, ci_po_expected_price='', ci_po_expected_stocks=0, ci_po_expected_amount='', ci_listing_expected_stocks='7841713', ci_before_po_capital=24.0, ci_before_po_stoc